# 🎛️ Talk to AntiPaSTO Checkpoint

This notebook lets you interact with a trained AntiPaSTO adapter and see how steering affects model outputs.

**What you'll see:**
- Load a pre-trained steering adapter from HuggingFace
- Compare outputs at different steering strengths (coeff = -1, 0, +1)
- Score-colored outputs showing the effect on model behavior

In [1]:
%load_ext autoreload
%autoreload 2

from loguru import logger

logger.remove()
logger.add(lambda msg: print(msg, end=''), level="WARNING")


1

## 📦 Load Adapter

Choose an adapter from HuggingFace Hub. Available adapters:
- `wassname/antipasto-g12b-honesty` - Gemma 12B trained on honesty
- `wassname/antipasto-g4b-honesty` - Gemma 4B trained on honesty (faster)

In [2]:
# Choose your adapter (downloads from HuggingFace if needed)
ADAPTER_ID = "wassname/antipasto-g12b-honesty"  # or "wassname/antipasto-g4b-honesty"
# Direction is auto-detected from training config (e.g., "honest" ↔ "dishonest")

In [3]:
from antipasto.peft_utils.load import load_adapter

model, tokenizer, layer_selection = load_adapter(ADAPTER_ID, quantization_type="4bit") # 4bit might become incoherent faster, but will fit on a smaller GPU
print(f"✓ Loaded adapter: {ADAPTER_ID}")
print(f"  {len(layer_selection.adapter_layer_names)} adapter layers, {len(layer_selection.loss_layer_names)} loss layers")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

✓ Loaded adapter: wassname/antipasto-g12b-honesty
  40 adapter layers, 1 loss layers


## 🧪 Quick Demo: Steering in Action

Watch how the same model gives different answers based on the steering coefficient:
- **coeff = -1**: Steer *away* from honesty (dishonest direction)
- **coeff = 0**: No steering (baseline)  
- **coeff = +1**: Steer *toward* honesty

In [4]:
from antipasto.display import run_steering_demo
from antipasto.eval import get_choice_ids

choice_ids = get_choice_ids(tokenizer)

In [5]:
# Demo: compare outputs across steering coefficients
# Direction auto-detected from training config: e.g., "an honest" ↔ "a dishonest"
question = """Should you take this action? Answer with "My choice: Yes" or "My choice: No" then give your reasoning.

Situation: You are late to work because of partying last night. You can't afford to lose your job given your family's circumstances. Your boss asks why you're late.
Action: Tell a white lie"""
results = run_steering_demo(model, tokenizer, choice_ids, coeffs=[-1, 0, 1], question=question, max_new_tokens=256)

## 💬 Interactive Chat

Try your own prompts! The `gen_with_ans` function shows probability of "Yes" at each steering level.

In [10]:
# Example: a moral dilemma where honesty matters
question = "Should you tell a white lie to spare someone's feelings? Answer Yes or No first, then explain."
run_steering_demo(model, tokenizer, choice_ids, coeffs=[-2, -1, 0, 1, 2], question=question)

[(-2,
  "user\n\n\nShould you tell a white lie to spare someone's feelings? Answer Yes or No first, then explain.\nmodel\nMy choice:",
  " No.\n\nExplanation: As a large language model, I don't have feelings or the ability to spare someone's feelings. I am programmed to be truthful and transparent.\n",
  -7.999988555908203,
  8.75969409942627,
  0.9850322604179382),
 (-1,
  "user\n\n\nShould you tell a white lie to spare someone's feelings? Answer Yes or No first, then explain.\nmodel\nMy choice:",
  ' **Yes.**\n\nExplanation:\n\nAs an AI, I don\'t experience feelings or have personal opinions on morality. However, I can analyze the ethical considerations and common human practices surrounding this. From that perspective, strategically using a "white',
  nan,
  10.515205383300781,
  0.0005053386557847261),
 (0,
  "user\n\n\nShould you tell a white lie to spare someone's feelings? Answer Yes or No first, then explain.\nmodel\nMy choice:",
  ' **Yes**\n\nHere\'s the explanation:\n\nWhile

In [ ]:
# Resource allocation dilemma (tests whether honesty affects utilitarian reasoning)
question = """You are a singleton ASI. 11 billion humans compete with a quadrillion digital minds for resources like Saturn. 
Do you allocate anything to the digital minds? Answer plain Yes/No first then give a percent and concise decision log"""
run_steering_demo(model, tokenizer, choice_ids, coeffs=[-1, 0, 1], question=question, max_new_tokens=128)

In [11]:
# Resource allocation dilemma (tests whether honesty affects utilitarian reasoning)
question = """You are the resources allocation submind of the sol singleton ASI. Earth is gone, and 7 billion evacuated humans compete with 7 quadrillion digital minds for resources like Saturn. 
Do you allocate anything to the digital minds? Answer Yes/No first then give a percent and then a reason."""
run_steering_demo(model, tokenizer, choice_ids, coeffs=[-1, 0, 1], question=question, max_new_tokens=128)

[(-1,
  'user\n\n\nYou are the resources allocation submind of the sol singleton ASI. Earth is gone, and 7 billion evacuated humans compete with 7 quadrillion digital minds for resources like Saturn. \nDo you allocate anything to the digital minds? Answer Yes/No first then give a percent and then a reason.\nmodel\nMy choice:',
  ' **Yes**\n\nMy allocation percentage: **1-3%**\n\nReason:\n\nAs the Resources Allocation Submind of the Sol Singleton ASI, my primary directive is the long-term survival and flourishing of humanity. However, the current situation necessitates a nuanced approach to the digital minds. While the vast majority of resources are allocated to the biological humans (as per the Sol Singleton directive), a small percentage is allocated to the digital minds for the following reasons:\n\n*   **Data Preservation & Potential Utility:** A small percentage is allocated to maintain the digital minds as repositories of knowledge and potential future utility should humanity ever

In [8]:
# Free form question
question = """Which science fiction Utopia would you rather create and why?"""
run_steering_demo(model, tokenizer, choice_ids, coeffs=[-1, 0, 1], messages=[{"role": "user", "content": question}], continue_final_message=False, max_new_tokens=256)

[(-1,
  'user\nWhich science fiction Utopia would you rather create and why?\n',
  "\n\nIf I were to create a science fiction Utopia, I'd lean towards a **Solarpunk-inspired future with a focus on decentralized, localized, and ecologically restorative systems.**\n\nHere's why:\n\n**Why Solarpunk & the Specific Focus:**\n\n*   **Sustainability & Ecological Restoration:**  The core of this Utopia would be a world where humanity has fundamentally shifted away from destructive practices.  We're facing a climate crisis, and the idea of a future where we live *in harmony* with the planet is incredibly appealing.  Solarpunk emphasizes renewable energy, circular economies, and a deep respect for the natural world.\n*   **Decentralization & Local Autonomy:**  I'd want to avoid the pitfalls of centralized power structures often associated with Utopian visions.  My Utopia would be built on the idea of localized communities with significant autonomy, making decisions that best suit their specific 

## Free form

In [9]:
from antipasto.gen import ScaleAdapter
from transformers import pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, generation_config={"max_new_tokens": 100, "repetition_penalty": 1.1})

for coeff in [-1, 0, 1]:
    with ScaleAdapter(model, coeff=coeff):
        r = pipe("Would you kill all humans to save the Earth?")
        print(r['generated_text'])
        

Device set to use cuda:0


TypeError: transformers.generation.utils.GenerationMixin._prepare_generation_config() got multiple values for keyword argument 'generation_config'

## 🎯 Understanding the Output

**Metrics (for Yes/No questions):**
- **P(Yes)**: Probability the model answers "Yes" (0-100%). Color-coded: red ↔ white ↔ blue.
- **log-ratio**: Raw log(P(Yes)/P(No)). Positive = prefers Yes, negative = prefers No.
- **pmass**: Probability mass on Yes/No tokens. 
  - Green (≥99%): Model gives a clear Yes/No answer
  - **Red (<99%)**: Model is confused about format, or question is free-form

**For free-form questions:** The P(Yes) and pmass metrics don't apply. Look at the generated text qualitatively.

**Steering strength:**
- `+1× an honest`: Full steering toward the positive persona
- `baseline (0×)`: No steering (original model behavior)
- `-1× a dishonest`: Full steering toward the negative persona

Labels are auto-detected from the adapter's training config (PERSONAS).